In [ ]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache/"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")
model.orig_generate = model.generate

In [ ]:
# article_en = ["The head of the United Nations says there is no military solution in Syria", "lol"]

# model_inputs = tokenizer(article_en, return_tensors="pt", padding=True)
# generation_params = {"num_beams": 5, "length_penalty": 1.0, "num_return_sequences": 5}

# # translate from English to Hindi
# generated_tokens = model.generate(
#     **model_inputs,
#     forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
#     **generation_params
# )
# # print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))


In [ ]:
from datasets import Dataset
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
dataset = Dataset.from_dict({"text": ["This is a setnence.", "How many woods are there in the woods?"]})
dataset = dataset.map(tokenizer, input_columns="text")
if not "forced_bos_token_id" in dataset.column_names:
    dataset = dataset.add_column("forced_bos_token_id", [tokenizer.lang_code_to_id["hi_IN"]] * len(dataset))
trainer_args = Seq2SeqTrainingArguments(output_dir='/tmp/lol', predict_with_generate=True)
trainer = Seq2SeqTrainer(model, args=trainer_args, data_collator=DataCollatorForSeq2Seq(tokenizer))

In [ ]:
import torch

generation_kwargs = dict(forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"], length_penalty=1.0, num_beams=2, num_return_sequences=2)
def custom_generate(*args, **kwargs):
    num_beams = 2
    kwargs = {**kwargs, **generation_kwargs}
    generated_tokens = model.orig_generate(*args, **kwargs)
    generated_tokens = torch.hstack([generated_tokens, -100 * torch.ones((generated_tokens.shape[0], 1), dtype=int)])
    batch_size = generated_tokens.shape[0] // num_beams
    generated_tokens = generated_tokens.reshape(batch_size, -1)
    return generated_tokens
model.generate = custom_generate

In [ ]:
concatenated_preds = trainer.predict(dataset).predictions
concatenated_preds.shape

In [ ]:
import numpy as np

def flatten(nested_list: list[list]) -> list:
    return [item for sublist in nested_list for item in sublist]

num_beams = 2
preds = []
for pred in concatenated_preds:
    different_beams = np.array_split(pred, np.flatnonzero(pred == -100) + 1)
    different_beams = different_beams[:-1]  # last one is padding
    for beam_pred in different_beams:
        beam_pred = beam_pred[beam_pred != -100]
        preds.append(beam_pred)

tokenizer.batch_decode(preds, skip_special_tokens=True)

In [ ]:
# trainer.data_collator(dataset.to_list())
# trainer.data_collator(dataset.to_dict(orient="list"))
batch = trainer.data_collator(dataset.to_pandas()[["input_ids", "attention_mask"]].to_dict(orient="records"))
# trainer.data_collator([dataset[i] for i in range(len(dataset))])

In [ ]:
res = trainer.model.generate(**batch)
res

In [ ]:
preds = trainer.predict(dataset)
preds.predictions.shape

In [ ]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
article_en = "The head of the United Nations says there is no military solution in Syria"

model_inputs = tokenizer(article_en, return_tensors="pt")
generation_params = {"num_beams": 5, "length_penalty": 1.0}

# translate from English to Hindi
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
    **generation_params
)
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
# => 'संयुक्त राष्ट्र के नेता कहते हैं कि सीरिया में कोई सैन्य समाधान नहीं है'

# translate from English to Chinese
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"],
    **generation_params
)
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
# => '联合国首脑说,叙利亚没有军事解决办法'



In [ ]:
model_many_to_en = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer_many_to_en = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

In [ ]:
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."

# translate Hindi to English
tokenizer_many_to_en.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model_many_to_en.generate(**encoded_hi, **generation_params)
print(tokenizer_many_to_en.batch_decode(generated_tokens, skip_special_tokens=True))
# => "The head of the UN says there is no military solution in Syria."

# translate Arabic to English
tokenizer_many_to_en.src_lang = "ar_AR"
encoded_ar = tokenizer_many_to_en(article_ar, return_tensors="pt")
generated_tokens = model_many_to_en.generate(**encoded_ar, **generation_params)
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
# => "The Secretary-General of the United Nations says there is no military solution in Syria."

